In [1]:
%%capture
!pip3 install google-auth google-cloud-dialogflow-cx



## imports

In [2]:
# dfcx-scrapi
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

# For connectopm to g-sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

# data manipulation
import pandas as pd
import numpy as np

# general python imports
import os

## g-sheets config

In [3]:
# service account scopes
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/christophertrauco/.config/secrets/cx_admin_sa.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '18qvh0uWkA0z7S5CC3xTdvD2wkhsmnINjgxUWmuoCo7M'
sh = client.open_by_key(spreadsheet_key)
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad


## master component ss id index

In [4]:
# object attribute mapping index
bc_sad_dfcx_master = sh.worksheet('Main').acell('B11').value
google_sheet_tab_read = sh.worksheet('Main').acell('B12').value



intents_ss = sh.worksheet('Index').acell('B1').value
print(intents_ss)

1lt02Hh2B_q_sluK-sAcgqyJ8WbqfZ6qDP_EQJVHPJDo


## agent object attribute mappings


In [16]:
# object data attribute mappings 
# master workbook @ https://itrau.co/bc-dfcx-sad-main

project_id = sh.worksheet('Main').acell('B2').value
agent_id = sh.worksheet('Main').acell('D3').value
display_name = sh.worksheet('Main').acell('B3').value
gcp_region = sh.worksheet('Main').acell('B4').value
zone = sh.worksheet('Main').acell('B5').value
language = sh.worksheet('Main').acell('B6').value
default_start_flow = sh.worksheet('Main').acell('B8').value
time_zone = sh.worksheet('Main').acell('B7').value
creds_path = sh.worksheet('Main').acell('B10').value
google_sheet_name = sh.worksheet('Main').acell('B11').value
google_sheet_tab_read = sh.worksheet('Main').acell('B12').value

# print(project_id, display_name, gcp_region, gcp_region, zone, language, default_start_flow,time_zone)

In [17]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds_path

In [18]:
parent_value = sh.worksheet('Main').acell('B13').value
print(parent_value)

projects/bot1-404120/locations/global/agents/4a2fde83-decc-4656-8995-1cbc2e3eeab4


# output selected agent intents

In [19]:
from google.cloud import dialogflowcx_v3


def list_intents():
    # Create a client
    client = dialogflowcx_v3.IntentsClient()

    # Initialize request argument(s)
    request = dialogflowcx_v3.ListIntentsRequest(
        parent=parent_value,
    )

    # Make the request
    page_result = client.list_intents(request=request)

    # Handle the response
    for response in page_result:
        print(response)

list_intents()

name: "projects/bot1-404120/locations/global/agents/4a2fde83-decc-4656-8995-1cbc2e3eeab4/intents/00000000-0000-0000-0000-000000000000"
display_name: "Default Intent"
training_phrases {
  parts {
    text: "Hello"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "heya"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "hello hi"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "howdy"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "hey there"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "hi there"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "greetings"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "hey"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "lovely day isn\'t it"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "I greet you"
  }
  repeat_count: 1
}
training_phrases {
  parts {
    text: "hello again"
  }
  repeat_count: 1
}
training_phrase

In [20]:
%%sh
./etl.sh

Traceback (most recent call last):
  File "/Users/christophertrauco/bc-records-ai-chatbot-v2/response.py", line 4, in <module>
    from google.cloud import dialogflowcx_v3beta1
  File "/Users/christophertrauco/bc-records-ai-chatbot-v2/venv/lib/python3.11/site-packages/google/cloud/dialogflowcx_v3beta1/__init__.py", line 21, in <module>
    from .services.agents import AgentsAsyncClient, AgentsClient
  File "/Users/christophertrauco/bc-records-ai-chatbot-v2/venv/lib/python3.11/site-packages/google/cloud/dialogflowcx_v3beta1/services/agents/__init__.py", line 16, in <module>
    from .async_client import AgentsAsyncClient
  File "/Users/christophertrauco/bc-records-ai-chatbot-v2/venv/lib/python3.11/site-packages/google/cloud/dialogflowcx_v3beta1/services/agents/async_client.py", line 31, in <module>
    from google.api_core import exceptions as core_exceptions
  File "/Users/christophertrauco/bc-records-ai-chatbot-v2/venv/lib/python3.11/site-packages/google/api_core/exceptions.py", line 

In [ ]:
df = pd.read_csv('intents.csv', on_bad_lines='skip')
# print(df)

wks_name = 'Intents'
cell_of_start_df = 'A1'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           intents_ss,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

In [ ]:
dffx = DataframeFunctions(creds_path)
# print(dffx)


In [ ]:
df = dffx.sheets_to_dataframe(bc_sad_dfcx_master, google_sheet_tab_read)

In [ ]:
dffx.bulk_update_intents_from_dataframe(agent_id, df, update_flag=True, rate_limiter=1)